In [3]:
import scipy.io
import numpy as np
from scopt import scopt
import portfolio as pr


In [4]:
problem_name = 'syn_1000_1500_10_50'
data = scipy.io.loadmat('../data/' + problem_name + '.mat')
R = data['W']
results = {problem_name: {}}

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3
sigma_f = 10
diam_X = np.sqrt(2)
rho = np.sqrt(n)

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [10]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad: pr.llo_oracle(x, r, grad, rho=rho)
prox_func = lambda x, L: pr.proj_simplex(x)

## Run FW with LLOO

In [6]:
run_alpha_policies = ["standard","line_search","icml"]
# run_alpha_policies = ['lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = FW.frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       llo_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x,
                       sigma_f=sigma_f,
                       diam_X=diam_X,                                            
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
iter = 1, stepsize = 0.6666666666666666, criterion = 0.6664444073950565, upper_bound=0.021289312869133274, lower_bound=-4.917163298674772
iter = 1000, stepsize = 0.001996007984031936, criterion = 0.0015571040685596691, upper_bound=-4.475083631695001, lower_bound=-4.475580917426577
iter = 2000, stepsize = 0.000999000999000999, criterion = 0.0007783134107878446, upper_bound=-4.475090642267503, lower_bound=-4.475279506200186
iter = 3000, stepsize = 0.0006662225183211193, criterion = 0.0005181100345580247, upper_bound=-4.4750919166298555, lower_bound=-4.475246190227915


KeyboardInterrupt: 

In [6]:
alpha_hist

[8.228559592572766e-11]

## Run SCOPT

In [5]:
 x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

iter =    1, stepsize = 5.663e-01, rdiff = 4.796e-01 , f = 0.0212893



KeyboardInterrupt: 